In [3]:
import matplotlib.pyplot as plt
import sys
import os
import pandas as pd
import shutil
import itertools
HOME_DIR = os.path.expanduser("~")
sys.path.append(f"{HOME_DIR}/spdk/exp-0219/plot")
from utils import *

In [5]:
res_dir = f"{HOME_DIR}/spdk/exp-0219/result"

keywords = ["64k", "OP_14", "zipf:1.1"]

result_files = []

for res_file in os.listdir(res_dir):
    flag = True
    for keyword in keywords:
        if keyword not in res_file:
            flag = False
    if flag:
        res_file_path = os.path.join(res_dir, res_file)
        result_files.append(res_file_path)

result_files.sort()

print(len(result_files))
print("\n".join(result_files))

IO_ITEMS = ["USER", "COMPACTION", "BASE_MD", "NVC_MD", "L2P"]

8
/users/guntherX/spdk/exp-0219/result/ALGO_mida22_cb_BS_64k_WP_rand_OP_14_DIS_zipf:1.1.job.out
/users/guntherX/spdk/exp-0219/result/ALGO_mida24_cb_BS_64k_WP_rand_OP_14_DIS_zipf:1.1.job.out
/users/guntherX/spdk/exp-0219/result/ALGO_mida44_cb_BS_64k_WP_rand_OP_14_DIS_zipf:1.1.job.out
/users/guntherX/spdk/exp-0219/result/ALGO_mida46_cb_BS_64k_WP_rand_OP_14_DIS_zipf:1.1.job.out
/users/guntherX/spdk/exp-0219/result/ALGO_sepbit22_cb_BS_64k_WP_rand_OP_14_DIS_zipf:1.1.job.out
/users/guntherX/spdk/exp-0219/result/ALGO_sepbit24_cb_BS_64k_WP_rand_OP_14_DIS_zipf:1.1.job.out
/users/guntherX/spdk/exp-0219/result/ALGO_sepbit44_cb_BS_64k_WP_rand_OP_14_DIS_zipf:1.1.job.out
/users/guntherX/spdk/exp-0219/result/ALGO_sepbit46_cb_BS_64k_WP_rand_OP_14_DIS_zipf:1.1.job.out


In [6]:
def plot_io(result_file, metric, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir, exist_ok=True)
    read_metric_dict = {}
    write_metric_dict = {}
    for item in IO_ITEMS:
        stat = extract_stat_io(result_file, item)
        read_metric_dict[item] = stat[f"read_{metric}"]
        write_metric_dict[item] = stat[f"write_{metric}"]

    read_metric_df = pd.DataFrame(read_metric_dict)
    write_metric_df = pd.DataFrame(write_metric_dict)
    read_metric_df.to_csv(os.path.join(save_dir, f"read_{metric}.csv"))
    read_metric_df = read_metric_df[int(0.1*read_metric_df.shape[0]):int(0.9*read_metric_df.shape[0])]
    plt.clf()
    plt.xlabel("Time (s)")
    if metric == "bw":
        plt.ylabel("Bandwidth (MB/s)")
    else:
        plt.ylabel("IOPS")
    read_metric_df.plot()
    plt.savefig(os.path.join(save_dir, f"read_{metric}.pdf"))
    write_metric_df.to_csv(os.path.join(save_dir, f"write_{metric}.csv"))
    write_metric_df = write_metric_df[int(0.1*write_metric_df.shape[0]):int(0.9*write_metric_df.shape[0])]
    plt.clf()
    plt.xlabel("Time (s)")
    if metric == "bw":
        plt.ylabel("Bandwidth (MB/s)")
    else:
        plt.ylabel("IOPS")
    write_metric_df.plot()
    plt.savefig(os.path.join(save_dir, f"write_{metric}.pdf"))
    

In [7]:
def plot_stat_waf(result_file, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir, exist_ok=True)
    stat = extract_stat_waf(result_file)
    stat_df = pd.DataFrame(stat)
    stat_df.to_csv(os.path.join(save_dir, "stat_waf.csv"))
    stat_df = stat_df[int(0.1*stat_df.shape[0]):int(0.9*stat_df.shape[0])]
    plt.clf()
    stat_df.plot()
    plt.xlabel("Time (s)")
    plt.ylabel("WAF")
    plt.savefig(os.path.join(save_dir, "stat_waf.pdf"))

In [8]:
def plot_overall_waf(result_file, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir, exist_ok=True)
    waf, final_waf = extract_overall_waf(result_file)
    stat_df = pd.DataFrame(waf)
    stat_df.to_csv(os.path.join(save_dir, "overall_waf.csv"))
    stat_df = stat_df[int(0.1*stat_df.shape[0]):int(0.9*stat_df.shape[0])]
    plt.clf()
    plt.xlabel("Time (s)")
    plt.ylabel("WAF")
    stat_df.plot()
    plt.savefig(os.path.join(save_dir, "overall_waf.pdf"))
    return waf, final_waf

In [ ]:
result_dics = []
for result_f in result_files:
    try:
        core_result_f = dump_core_log(result_f, section=3)
        f = result_f.split(os.sep)[-1].split(".job.out")[0]
        plot_dir = "/home/xgj/spdk/exp-0219/plot/result"
        if not os.path.exists(plot_dir):
            os.mkdir(plot_dir)
        save_dir = os.path.join(plot_dir, f)
        for metric in ["IOPS", "bw"]:
            plot_io(core_result_f, metric, save_dir)
        plot_stat_waf(core_result_f, save_dir)
        _, final_waf = plot_overall_waf(core_result_f, save_dir)
        iops, bw, latency = extract_fio(result_f)
        result_dics.append({
            "param": f,
            "IOPS": iops,
            "BW(MB/S)": bw,
            "latency": latency,
            "final_waf": final_waf,
        })
    except Exception as e:
        print(f"Error: {e}")
    finally:
        os.remove(core_result_f)
result_df = pd.DataFrame(result_dics)
print(result_df)
result_df.to_csv(os.path.join(f"{HOME_DIR}/spdk/exp-0219/result", f"{'-'.join(keywords)}.csv"), sep="\t")

Error: [Errno 2] No such file or directory: '/home/xgj/spdk/exp-0219/plot/result'
Error: [Errno 2] No such file or directory: '/home/xgj/spdk/exp-0219/plot/result'
Error: [Errno 2] No such file or directory: '/home/xgj/spdk/exp-0219/plot/result'
Error: [Errno 2] No such file or directory: '/home/xgj/spdk/exp-0219/plot/result'
Error: [Errno 2] No such file or directory: '/home/xgj/spdk/exp-0219/plot/result'
Error: [Errno 2] No such file or directory: '/home/xgj/spdk/exp-0219/plot/result'


Error: [Errno 2] No such file or directory: '/home/xgj/spdk/exp-0219/plot/result'
Error: [Errno 2] No such file or directory: '/home/xgj/spdk/exp-0219/plot/result'
Empty DataFrame
Columns: []
Index: []


OSError: Cannot save file into a non-existent directory: '{HOME_DIR}/spdk/exp-0219/result'